## 1. What is the Vehicle Routing Problem (VRP)?
The Vehicle Routing Problem (VRP) is a classical combinatorial optimization problem that involves determining the optimal set of routes for a fleet of vehicles to serve a given set of customers, with the objective of minimizing the total cost or distance traveled while satisfying certain constraints.

The key points of the VRP are:

- Depot: A location where all vehicles start and come back after doing their routes.
- Customers: Each customer got his locations. Customers add specific **constraints** that need to be respected like the time when there are available to receive the items.
- Vehicles: Each vehicle has a **limited capacity** (weight). Moreover there are multiple vehicles. 
- Costs: Various cost factors play a role in the VRP, including travel distances, vehicle capacities, time constraints, etc...

## 2. Solving the Vehicle Routing Problem
Knowing that the problem is NP-Complete, we can't solve it to find the optimal answer in a polynomial time. Beacause of that, we had to use **heuristic methods** which will allow us have an acceptable answer but not the ideal one.\\
For each algorithm, we will use that dataset :
#### Dataset

| Location | X  | Y  | Demand |
|----------|----|----|--------|
| 0        | 0  | 0  | 0      |
| 1        | 6  | 7  | 58     |
| 2        | 1  | 7  | 36     |
| 3        | 5  | 3  | 15     |
| 4        | 2  | 4  | 68     |
| 5        | 2  | 5  | 91     |
| 6        | 2  | 1  | 40     |
| 7        | 2  | 9  | 11     |
| 8        | 9  | 5  | 83     |
| 9        | 2  | 9  | 73     |
| 10       | 9  | 7  | 72     |
| 11       | 7  | 9  | 24     |
| 12       | 8  | 4  | 66     |
| 13       | 4  | 5  | 65     |
| 14       | 7  | 3  | 93     |
| 15       | 10 | 10 | 85     |
| 16       | 9  | 5  | 22     |
| 17       | 2  | 4  | 18     |
| 18       | 7  | 3  | 94     |
| 19       | 1  | 5  | 58     |
| 20       | 4  | 6  | 25     |

#### The label's columns stand for: 
- X is the x-coordinate of the customer location
- Y is the y-coordinate of the customer location
- Demand is the quantity that needs to be delivered

### 2.1 Heuristic Algorithm
Heuristic algorithms are practical techniques used to find good-enough solutions for optimization problems, especially when finding the exact solution is computationally expensive or infeasible. These algorithms rely on problem-specific knowledge or rules of thumb to guide the search process, sacrificing optimality guarantees for faster computation.

**Key Characteristics of Heuristic Algorithms**

**Fast and Efficient**: Heuristics prioritize speed over precision, making them suitable for large or time-sensitive problems.
**Problem-Specific**: They often incorporate domain-specific insights to improve performance on particular problem types.
**No Guarantee of Optimality**: While heuristics often find satisfactory solutions, they do not guarantee that the best solution will be found.

**Limitations**
Heuristic methods can quickly produce solutions but are prone to getting stuck in local optima. For this reason, they are often complemented by metaheuristic approaches to achieve better results on complex problems.

### 2.1 a) Nearest Neighbor Algorithm
The Nearest Neighbor algorithm is a simple and intuitive heuristic that constructs routes by iteratively choosing the **nearest customer** to the current customer. The algorithm starts with an empty route for each vehicle and an unvisited customer list. It then selects a starting customer (usually the depot) and iteratively adds the nearest unvisited customer to the current route until all customers are visited. The Nearest Neighbor algorithm is known for its efficiency and ease of implementation. However, it may not always produce optimal solutions as it can lead to suboptimal routes in certain scenarios.

In [1]:
#import libraries
import openpyxl
import numpy as np
import pandas as pd

def read_excel_file(filename, sheet_name):
    """
    Read coordinates and demand values from a specific sheet in an Excel file.
    Assumes the data is in columns labeled 'X', 'Y', and 'Demand'.
    """
    df = pd.read_excel(filename, sheet_name=sheet_name)
    coordinates = df[['X', 'Y']].values
    demands = df['Demand'].values
    return coordinates, demands


def adjacency_matrix(coordinates):
    """
    Calculate the adjacency matrix.
    """
    num_points = len(coordinates)
    adj_matrix = np.zeros((num_points, num_points))

    for i in range(num_points):
        for j in range(num_points):
            adj_matrix[i, j] = calculate_distance(coordinates, i, j)
    return adj_matrix



def calculate_distance(coordinates, i, j):
    """
    Calculate the Euclidean distance between two points.
    """
    x1, y1 = coordinates[i]
    x2, y2 = coordinates[j]
    return np.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)


def calculate_total_distance(route, adj_matrix):
    """
    Calculate the total distance of a given route using the distance matrix.
    """
    total_distance = 0
    num_points = len(route)

    for i in range(num_points - 1):
        current_node = route[i]
        next_node = route[i + 1]
        total_distance += adj_matrix[current_node, next_node]

    return total_distance


def nearest_neighbor(adj_matrix, demands, capacity):
    """
    Apply the Nearest Neighbor heuristic to find initial routes for VRP.
    """
    num_points = adj_matrix.shape[0]
    visited = [False] * num_points   # List of boolean to know if a node is visited or not
    routes = []

    while sum(visited) < num_points:
        current_node = 0  # Start at node 0
        current_capacity = 0
        route = [current_node]
        visited[current_node] = True

        while current_capacity + demands[current_node] <= capacity:
            last_node = route[-1] 
            nearest = None           # Initialize nearest the None
            min_dist = float('inf')  # Initialize min_dist with the higher number possible to not bias routes

            for neighbor in [i for i, v in enumerate(visited) if not v]:  # Loop through unvisited nodes
                if demands[neighbor] + current_capacity <= capacity and adj_matrix[last_node, neighbor] < min_dist:
                    nearest = neighbor
                    min_dist = adj_matrix[last_node, neighbor]

            if nearest is None:
                break

            route.append(nearest)
            visited[nearest] = True
            current_capacity += demands[nearest]

        routes.append(route)

    return routes



def format_output(routes):
    """
    Format the final routes as required.
    In this example, it returns a list of routes.
    """
    routes = [[int(node) for node in route] for route in routes]
    return routes


def vrp_solver(filename, sheet_name, capacity):
    """
    Solve the VRP using the provided filename for coordinates and vehicle capacity.
    """
    coordinates, demands = read_excel_file(filename, sheet_name)
    adj_matrix = adjacency_matrix(coordinates)
    routes = nearest_neighbor(adj_matrix, demands, capacity)
    final_routes = format_output(routes)
    return final_routes

#Use nearest neighbor
filename = "dataset.xlsx"
sheet_name = "Feuil1"  # Specify the name of the sheet or its index
capacity = 200 # Specify the capacity of the vehicle
solution = vrp_solver(filename, sheet_name, capacity)

for route in solution:  
    print(route)
    
    

[0, 6, 4, 17, 19, 7]
[0, 5, 13, 20, 3]
[0, 2, 9, 1, 11]
[0, 14, 18]
[0, 12, 8, 16]
[0, 10, 15]


### 2.1 b) Two-Opt Algorithm

The Two-Opt algorithm is a local search heuristic that seeks to optimize existing routes by iteratively removing two edges from the routes and reconnecting them differently, thereby reducing the total distance or cost. The algorithm starts with an initial solution, such as those generated by Nearest Neighbor or the Swap Method. It repeatedly applies the two-opt move until no further improvement is possible. The Two-Opt algorithm is relatively simple and can yield significant improvements over initial solutions.

In [2]:
#Use two opt
def two_opt(routes, adj_matrix, num_iterations):
    best_routes = routes.copy()

    for _ in range(num_iterations):
        selected_route_idx = np.random.randint(0, len(routes))
        selected_route = routes[selected_route_idx]

        i, j = np.random.randint(1, len(selected_route) - 1, size=2)
        if j < i:
            i, j = j, i

        new_route = selected_route.copy()
        new_route[i:j] = selected_route[j - 1: i - 1: -1]  # Reverse the path between i and j

        new_routes = routes.copy()
        new_routes[selected_route_idx] = new_route

        if calculate_total_distance(new_routes[selected_route_idx], adj_matrix) < calculate_total_distance(
                best_routes[selected_route_idx], adj_matrix
        ):
            best_routes = new_routes

    return best_routes

def vrp_solver2(filename, sheet_name, capacity, num_iterations):
    """
    Solve the VRP using the provided filename for coordinates, vehicle capacity,
    and number of iterations for the two-opt optimization.
    """
    coordinates, demands = read_excel_file(filename, sheet_name)
    adj_matrix = adjacency_matrix(coordinates)
    routes = nearest_neighbor(adj_matrix, demands, capacity)

    for i in range(len(routes)):
        route = routes[i]
        optimized_route = two_opt([route], adj_matrix, num_iterations)[0]
        routes[i] = optimized_route

    formatted_routes = format_output(routes)
    return formatted_routes

filename = "dataset.xlsx"
sheet_name = "Feuil1"  # Specify the name of the sheet or its index
capacity = 200 # Specify the capacity of the vehicle
num_iterations = 100000
solution=vrp_solver2(filename, sheet_name, capacity, num_iterations)

for route in solution:
    print(route)

[0, 6, 4, 17, 19, 7]
[0, 5, 13, 20, 3]
[0, 2, 9, 1, 11]
[0, 14, 18]
[0, 12, 8, 16]
[0, 10, 15]


The two-opt optimization will further improve the initial routes obtained using the Nearest Neighbor heuristic by iteratively swapping segments of the routes to minimize the total distance traveled. The number of iterations for two-opt is determined by the num_iterations parameter. The more iterations you use, the better the optimization and the longer the execution time. Adjust it according to your requirements.

### 2.1 c) Hill Climbing Algorithm

Hill Climbing is a heuristic search algorithm used primarily for mathematical optimization problems in artificial intelligence (AI). It is a form of local search, which means it focuses on finding the optimal solution by making incremental changes to an existing solution and then evaluating whether the new solution is better than the current one. The process is analogous to climbing a hill where you continually seek to improve your position until you reach the top, or local maximum, from where no further improvement can be made.

Hill climbing is a fundamental concept in AI because of its simplicity, efficiency, and effectiveness in certain scenarios, especially when dealing with optimization problems or finding solutions in large search spaces.

Hill climbing follows these steps:

- Initial State: Start with an arbitrary or random solution (initial state).
- Neighboring States: Identify neighboring states of the current solution by making small adjustments (mutations or tweaks).
- Move to Neighbor: If one of the neighboring states offers a better solution (according to some evaluation function), move to this new state.
- Termination: Repeat this process until no neighboring state is better than the current one. At this point, you’ve reached a local maximum or minimum (depending on whether you’re maximizing or minimizing).

#### This algorithm will be see in the 4<sup>th</sup> prosit called: <b>NO FREE LUNCH</b>

### 2.2 Metaheuristic algorithm

In our project, we are shifting our focus towards metaheuristic algorithms. While the previously discussed algorithms were heuristics, metaheuristics provide a more sophisticated approach for solving complex optimization problems, especially for NP-complete problems like ours.

Metaheuristics combine both intensification and diversification strategies, improving the likelihood of finding high-quality solutions. This distinguishes them from simpler heuristics, like the Hill Climbing, which can easily get stuck in local optima. By exploring larger solution spaces and introducing randomness, metaheuristics have the potential to explore global optima, even for very large or difficult problems.

#### Types of Algorithms

![](../illustrations/types_of_algo.png)

**Hypotesis** : Maybe using a heuristic algorithm to set the starting point of a metaheuristic algorithm can reduce execution time to get an acceptable answer and make this answer bettter.

#### Types of Metaheuristic Algorithms
Metaheuristic algorithms can be broadly categorized based on their approach to exploring the solution space. The two primary classifications are trajectory-based methods and population-based methods, each with its unique strategies and advantages.

**1. Trajectory-Based Metaheuristics**

These algorithms focus on a single solution and iteratively improve it by exploring its neighborhood. They are well-suited for problems where local refinement plays a significant role.

Simulated Annealing (SA): Inspired by the annealing process in metallurgy, this algorithm introduces randomness to escape local optima by allowing occasional worse moves, with the probability decreasing over time.
Tabu Search (TS): Uses memory structures to avoid revisiting previously explored solutions, thus diversifying the search process and escaping cycles.
Variable Neighborhood Search (VNS): Systematically changes the neighborhood structure during the search to explore diverse regions of the solution space.

**2. Population-Based Metaheuristics**

These methods maintain a set of solutions and improve them by leveraging interactions within the population. They are particularly powerful for problems requiring global exploration.

Genetic Algorithms (GA): Inspired by natural evolution, this algorithm uses operators like selection, crossover, and mutation to evolve a population of solutions toward better results.
Particle Swarm Optimization (PSO): Models the behavior of swarms (e.g., birds or fish) to iteratively refine solutions based on their own experience and that of their neighbors.
Ant Colony Optimization (ACO): Mimics the foraging behavior of ants, where pheromone trails guide the search toward promising solutions.

**3. Hybrid Metaheuristics**

Some approaches combine elements of different metaheuristics or mix heuristic methods with metaheuristics to exploit their complementary strengths. Examples include:

GRASP (Greedy Randomized Adaptive Search Procedure): Alternates between constructing initial solutions with randomness and refining them using local search.
Memetic Algorithms: Combines genetic algorithms with local search techniques for more intense exploitation of promising regions.

**4. Other Emerging Approaches**

Artificial Bee Colony (ABC): Based on the foraging behavior of honeybees, focusing on exploring and exploiting the search space effectively.
Harmony Search (HS): Inspired by musical improvisation, this algorithm fine-tunes solutions by balancing exploration and exploitation.
Choosing the Right Metaheuristic
The choice of metaheuristic depends on the problem's characteristics, such as the solution space's complexity, constraints, and the need for balancing intensification (exploiting good solutions) and diversification (exploring new areas).

## 3. To resume


Genetic algorithms, simulated annealing, and PSO (Particle swarm optimization) are all metaheuristic algorithms that can explore the search space and find good solutions to optimization problems. But, they differ in their approach and implementation, and are better suited for different types of problems.
- Genetic algorithms are good at handling complex optimization problems with many variables and constraints, but can be slow to converge.
- Simulated annealing is well-suited for finding the global optimum in a large search space with many local optima, but may converge slowly and requires careful tuning of the cooling schedule.
- Particle swarm optimization is good at handling nonlinear and dynamic optimization problems, but may be prone to premature convergence and requires careful selection of parameters.

https://www.youtube.com/watch?v=dNNZxV0RQsw

### <b>Thanks to the 4<sup>th</sup> prosit, we will be able to find the best algorithm for the project</b>